# Part 5 - Youth Sub Wallet Management

## 1. Initialise a controller for Yoma Youth Sub Wallet 1

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://yoma-multi-tenant-agent:8021"

# JWT KEY for Youth 1 Sub Wallet
%store -r youth1_token

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, jwt_token=youth1_token)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fb25c385790>


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 8a. Check for any existing connections

In [3]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS


### 8. Paste the invitation from the Yoma Agent Issuer notebook into the invitation variable here

In [4]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '28cd68f5-efa9-40d2-9e35-48e4b2b8bb9a', 'label': 'EXTERNAL', 'serviceEndpoint': 'https://cefae0076989.ngrok.io', 'recipientKeys': ['9LJfrZayMeD3FwCZiLBav6qsspgJZK8uT8Gf3xrptigW']}

### 9. Accept the invitation, then move to [Yoma Agent Issuer notebook](http://localhost:8888/notebooks/Part%204%20-%20Yoma%20Agent%20Credential%20Issuance.ipynb)

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
connection_id = response["connection_id"]
STATE = response["state"]

Connection {'created_at': '2021-03-16 15:29:36.698467Z', 'invitation_mode': 'once', 'connection_id': '98e7a2a3-80d9-4dad-9efa-136e034f4982', 'routing_state': 'none', 'updated_at': '2021-03-16 15:29:36.762806Z', 'my_did': 'QkYewzpU8qMDf2VsPWZRp2', 'state': 'request', 'invitation_key': '9LJfrZayMeD3FwCZiLBav6qsspgJZK8uT8Gf3xrptigW', 'request_id': '360c3041-edf8-46cb-927e-31357d2ae02b', 'accept': 'manual', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'rfc23_state': 'request-sent'}


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [6]:
# Print connection list
connection = await agent_controller.connections.get_connection(connection_id)
print("Youth Wallet 1 AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Youth Wallet 1 AGENT CONNECTION
{'created_at': '2021-03-16 15:29:36.698467Z', 'invitation_mode': 'once', 'connection_id': '98e7a2a3-80d9-4dad-9efa-136e034f4982', 'routing_state': 'none', 'updated_at': '2021-03-16 15:29:40.091954Z', 'my_did': 'QkYewzpU8qMDf2VsPWZRp2', 'state': 'active', 'invitation_key': '9LJfrZayMeD3FwCZiLBav6qsspgJZK8uT8Gf3xrptigW', 'request_id': '360c3041-edf8-46cb-927e-31357d2ae02b', 'accept': 'manual', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'rfc23_state': 'completed', 'their_did': 'Sdw8onVcs7QsroeU4Yy9TG'}
State: active


## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.


In [7]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")



Credential exchange a812cbaf-ed13-4d04-9dfd-8ced099172e4, role: holder, state: request_sent
Being offered: [{'name': 'fullname', 'value': 'Lohan Spies'}, {'name': 'mobile', 'value': '0832582698'}, {'name': 'email', 'value': 'eax@eax.co.za'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state


In [8]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange a812cbaf-ed13-4d04-9dfd-8ced099172e4, role: holder, state: credential_acked


## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [9]:
response = await agent_controller.terminate()
print(response)

None


# Proceed to Part 6 on [Opportunity Provider Onboarding notebook](http://localhost:8888/notebooks/Part%206%20-%20Yoma%20Opportunity%20Provider%20Onboarding%20and%20Verification.ipynb)

Here you will present the attributes issued to you within this tutorial to a verifying entity.